**Morphology exporter**

Hi!

This notebook aims to explain the SWC to NeuroML converter I made.
If you have any questions regarding the conversion, please contact me at s.reissenweber12@gmail.com.

The converter translates the points in the SWC file to NeuroML segments, and organizes them into unbranched segment groups based on their type.
These segment groups are then included in main segment groups (e.g. axon_group, soma_group).

The converter tries to keep as much of the provided information as possible. For instance, comments at the start of SWC files, which often contain useful information, are preserved as notes at the beginning of the generated NML document.

The converter also checks for SWC files that would be invalid in NeuroML and raises an exception with an error message when it encounters such issues. 
One common problem is loops in the SWC file, which are not valid for NeuroML simulations. The code identifies these loops and removes the segment groups containing them.

Good luck with understanding the code! :)


In [18]:
# Necessary imports:
import neuroml
import neuroml.writers as writers
import re
import os
import pprint

In [19]:
def construct_nml(path, output_dir=''):
    '''
    This function is the big function that calls all helper functions to construct the neuroml file.

    Input: - path: filepath to SWC file (str)
           - output_dir (optional): directory in which the neuroml file will be saved (str)

    Returns: name of the newly created neuroml file (str)
    '''
    
    errors = {}
    d, comments = open_and_split(path, errors)
    filename = os.path.basename(path).split('.')[0]
    filename = change_filename(filename, errors)
    cell_ID = f"{filename}_cell"
    nml_doc = neuroml.NeuroMLDocument(id=filename)
    nml_cell = neuroml.Cell(id=cell_ID)

    make_notes(comments, nml_cell)
    n, children, type_seg, root = classify_types_branches_and_leafs(d, errors)
    segmentGroups = find_segments(d, n, cell_ID, errors)
    nml_mor = process_segments(d, children, root, cell_ID, errors)
    nml_cell = process_cables(segmentGroups, type_seg, nml_mor, nml_cell, errors)
    nml_cell = define_biophysical_properties(nml_cell, cell_ID)

    nml_doc.cells.append(nml_cell)
    
    if output_dir:
        nml_file = f'{output_dir}/{filename}_converted.cell.nml'
    else:
        nml_file = f'{filename}_converted.cell.nml'
    writers.NeuroMLWriter.write(nml_doc, nml_file)
    pprint.pprint(errors)

    return nml_file.split('/')[-1]

In [20]:
def log_error(errors, error_type, occurrence=1, extra_info=None, fix=None, stop=False):
    # Check if error_type is related to unknown SWC types
    if error_type.startswith("Unknown type detected"):
        type_id = error_type[23:]
        if "Unknown type detected" not in errors:
            errors["Unknown type detected"] = {}

        if type_id not in errors["Unknown type detected"]:
            errors["Unknown type detected"][type_id] = {
                "occurrences": 0,
                "extra_info": [],
                "fix": None
            }

        errors["Unknown type detected"][type_id]["occurrences"] += occurrence
        if extra_info is not None:
            errors["Unknown type detected"][type_id]["extra_info"].append(extra_info)
        if fix is not None:
            errors["Unknown type detected"][type_id]["fix"] = fix
    else:
        if error_type not in errors:
            errors[error_type] = {
                "occurrences": 0,
                "extra_info": [],
                "fix": None
            }

        errors[error_type]["occurrences"] += occurrence
        if extra_info is not None:
            errors[error_type]["extra_info"].append(extra_info)
        if fix is not None:
            errors[error_type]["fix"] = fix

    if stop:
        pprint.pprint(errors)
        raise Exception(error_type)


In [21]:
def change_filename(filename, errors):
    new_name = re.sub(r'[^a-zA-Z0-9_]', '_', filename)
    if new_name[0].isdigit():
        new_name = '_' + new_name
    print(filename, new_name)
    if new_name != filename:
        log_error(errors, "Filename does not comply with neuroml filename pattern restrictions.", fix=f"Changed filename to {filename}")
        
    return new_name

In [22]:
def open_and_split(path, errors):
    '''
    This function takes a (path to an) SWC file and creates a dictionary with necessary information to generate the neuroml file.
    
    Input: path: filepath to SWC file (str)
    
    Returns: - d: dictionary {point (int): (type, x_coord, y_coord, z_coord, radius, parent)}
             - comments: list of comments [comment (str)]
    '''

    d = {}
    line_nr = 0
    comments = []
    no_par = [0, []]  # Variable used for checking amount of segments without parent

    with open(path, 'r+') as f:
        for line in f:
            line_nr += 1
            if not line:
                pass
            elif line[0] == '#':
                comments.append(line[1:].strip())
            else:
                information = [elem for elem in line.strip().split(' ') if elem]
                if not information:
                    pass
                else:
                    if len(information) != 7:
                        log_error(errors, "Line in SWC file contains an invalid amount of columns (more or less than 7)", extra_info=line_nr, fix="Skipped these lines")
                    else:
                        seg_ID = int(information[0]) - 1
                        type_ID = int(information[1])
                        x_coor = float(information[2])
                        y_coor = float(information[3])
                        z_coor = float(information[4])
                        rad = float(information[5])
                        par_ID = int(information[6]) - 1

                        if par_ID < 0:
                            par_ID = -1
                            no_par[0] += 1
                            no_par[1].append(seg_ID)

                        d[seg_ID] = (type_ID, x_coor, y_coor, z_coor, rad, par_ID)
        
    # Check if cell has segments
    if not d:
        log_error(errors, "SWC file does not contain any segments", fix="No fixes. SWC file is invalid.", stop=True)
    
    # Check if cell has more than one or zero segment(s) without a parent
    if no_par[0] == 0:
        log_error(errors, "Zero segments without parent (root segments) detected", fix="No fixes. SWC file is invalid.", stop=True)
    if no_par[0] > 1:
        log_error(errors, "More than one segment without parent (root segment) detected", extra_info=f"Points {no_par[1]}", fix="No fixes. SWC file is invalid.", stop=True)

    return d, comments

In [23]:
def make_notes(comments, nml_cell):
    '''
    This function creates the notes listed at the top of the neuroml file. It also includes the original comments listed in the SWC file.

    Input: - comments: list of comments [comment (str)]
           - nml_cell: neuroml cell object
    
    Returns: None
    '''

    nml_cell.notes = "\n\n" + '*' * 40 + \
                     "\nThis NeuroML file was converted from SWC to NeuroML format by Sietse Reissenweber's converter. \
                     \nFor any questions regarding the conversion, you can email me at s.reissenweber12@gmail.com. \
                     \nThe notes listed below are the notes that were originally contained in the SWC file.\n" \
                     + '*' * 40 + "\n\n"

    nml_cell.notes += "#" * 40 + "\n\n"

    for comment in comments:
        nml_cell.notes += f'{comment}\n'

    nml_cell.notes += "\n" + "#" * 40 + "\n\n"

In [24]:
def classify_types_branches_and_leafs(d, errors):
    '''
    This function classifies the segments into different types, and determines the children of points.
    
    Input: d: dict {point (int): (type, x_coord, y_coord, z_coord, radius, parent)}
    
    Returns: - n: dict {amount of children (int): [points]}
             - children: dict {point (int): [children]}
             - type_seg: dict {point (int): type morph. part (e.g. soma) (str)}
             - root: point without parent (int)
    '''

    n = {0: [],
         1: [],
         2: []}
    root = -float("Inf")
    children = {}
    type_seg = {}

    for point, info in d.items():
        # Create dict n:
        number_of_children = 0
        for info2 in d.values():
            if info2[5] == point:
                number_of_children += 1
        if number_of_children == 0:
            n[0].append(point)
        elif number_of_children == 1:
            n[1].append(point)
        else:
            n[2].append(point)

        # Check for 0.0 diameter:
        if info[4] <= 0.0:
            d[point] = info[:4] + (0.000001,) + (info[5],)
            if point in n[0]:
                log_error(errors, "Endpoint of zero radius detected", extra_info=f"Point {point}", fix=f"Changed radius to small number {0.000001}")
            else:
                log_error(errors, "Internal point of zero radius detected", extra_info=f"Point {point}", fix=f"Changed radius to small number {0.000001}")

        # Create dicts type_seg and types:
        if info[0] == 1:
            type_seg[point] = 'soma'
        elif info[0] == 2:
            type_seg[point] = 'axon'
        elif info[0] == 3:
            type_seg[point] = 'bas_dend'
        elif info[0] == 4:
            type_seg[point] = 'ap_dend'
        else:  # Account for custom types
            type_id = f'custom_{info[0]}'
            type_seg[point] = type_id
            log_error(errors, f"Unknown type detected: {type_id}", fix=f"Added new type {type_id} and new group {type_id}_group")

        # Find root:
        if info[5] == -1:
            root = point
            if type_seg[root] != 'soma':
                log_error(errors, "Spherical root segment does not belong to soma_group", fix="No fixes. SWC file is invalid.", stop=True)

        children[point] = []

    # Create dict children:
    for point, info in d.items():
        if point != root:
            children[info[5]].append(point)

    return n, children, type_seg, root

In [25]:
def find_segments(d, n, cell_ID, errors):
    '''
    This function organizes the segments into unbranched segment groups of the same type.
    
    Input: - d: dict {point (int): (type, x_coord, y_coord, z_coord, radius, parent)}
           - n: dict {amount of children (int): [points]}
           - cell_ID: unique ID of neuroml cell (str)
           - children: dict {point (int): [children]}

    Returns: - segmentGroups: list with lists of segmentgroups [[point], [point], ...]
    '''

    segmentGroups = []

    # Processing from leaf points to branch points:
    for leaf in n[0]:
        toAdd = leaf
        group_type = d[toAdd][0]
        segGr = []
        segmentFound = False

        if toAdd == 0:
            segmentFound = True
            segGr.append(toAdd)

        while segmentFound is False:
            if toAdd == 0 and toAdd in n[2]:  # Start new segmentgroup at branching point, even if root
                segmentFound = True
            elif toAdd == 0:
                segmentFound = True
                segGr.append(toAdd)
            elif toAdd in n[2]:  # Found a branch point
                segmentFound = True
            elif d[toAdd][0] != group_type:
                segmentGroups.append(segGr)
                segGr = []
                segGr.append(toAdd)
                group_type = d[toAdd][0]
                toAdd = d[toAdd][5]
            else:
                segGr.append(toAdd)
                toAdd = d[toAdd][5]

        if segGr:
            segmentGroups.append(segGr)
    
    # Processing from branch points to other branch points:
    for branch in n[2]:
        toAdd = branch
        group_type = d[toAdd][0]
        segGr = []
        segmentFound = False

        if toAdd == 0:
            segmentFound = True
            segGr.append(toAdd)

        while segmentFound is False:
            if toAdd == 0 and toAdd in n[2]: 
                segmentFound = True
            elif toAdd == 0:
                segmentFound = True
                segGr.append(toAdd)
            elif toAdd in n[2] and toAdd != branch:
                segmentFound = True
            elif d[toAdd][0] != group_type:
                segmentGroups.append(segGr)
                segGr = []
                segGr.append(toAdd)
                group_type = d[toAdd][0]
                toAdd = d[toAdd][5]
            else:
                segGr.append(toAdd)
                toAdd = d[toAdd][5]

        if segGr:
            segmentGroups.append(segGr)

    # Calculate amount of points processed
    N = 0
    for seggroup in segmentGroups:
        N += len(seggroup)

    if N != len(d):
        print("Number of processed segments:", N)
        print("Number of segments expected:", len(d))
        print("Watch out! Number of processed segments does not match.")
        print(f"The segments that are absent in the processed neuron {cell_ID} are:")
        non_present_segments = []
        for point in d:
            found = False
            for seggr in segmentGroups:
                for seg in seggr:
                    if seg == point:
                        found = True
            if found is False:
                non_present_segments.append(point)
        print(*non_present_segments)

    return segmentGroups

In [26]:
def process_segments(d, children, root, Cell_ID, errors):
    '''
    This function incorporates the segments into the neuroml morphology object.

    Input: - d: dict {point (int): (type, x_coord, y_coord, z_coord, radius, parent)}
           - children: dict {point (int): [children]} 
           - root: point without parent (int)
           - cell_ID: unique ID of neuroml cell (str) 

    Returns: nml_mor: neuroml morphology object    
    '''

    nml_mor = neuroml.Morphology(id=f'{Cell_ID}_morphology')

    available_points = [root]
    processed = []
    all_processed = False

    while all_processed is False:
        next_to_process = min(available_points)

        if next_to_process == root:  # Set distal and proximal points to root point if root
            Soma_Root = neuroml.Point3DWithDiam(x=str(d[next_to_process][1]),
                                                y=str(d[next_to_process][2]),
                                                z=str(d[next_to_process][3]),
                                                diameter=str(d[next_to_process][4] * 2))
            distalp = Soma_Root
            proximalp = Soma_Root
        else:
            distalp = neuroml.Point3DWithDiam(x=str(d[next_to_process][1]),
                                              y=str(d[next_to_process][2]),
                                              z=str(d[next_to_process][3]),
                                              diameter=str(d[next_to_process][4] * 2))
            parent = d[next_to_process][5]
            proximalp = neuroml.Point3DWithDiam(x=str(d[parent][1]),
                                                y=str(d[parent][2]),
                                                z=str(d[parent][3]),
                                                diameter=str(d[parent][4] * 2))

        parentID = d[next_to_process][5]
        if parentID != -1:
            # Only one segment may be spherical and must belong to the soma_group SegmentGroup:
            coord_distal = (d[next_to_process][1], d[next_to_process][2], d[next_to_process][3])
            coord_proximal = (d[parent][1], d[parent][2], d[parent][3])
            if coord_distal == coord_proximal and d[next_to_process][4] == d[parent][4]:
                log_error(errors, "Two segments detected with same radius and coordinates", extra_info=f"Segments {next_to_process} and {parent}", fix="No fix cause don't know what to change!!!!!")
            
            segpar = neuroml.SegmentParent(segments=parentID)
            thisSeg = neuroml.Segment(id=str(next_to_process),
                                      name=f'Comp_{str(next_to_process)}',
                                      distal=distalp,
                                      parent=segpar)
        else:
            thisSeg = neuroml.Segment(id=str(next_to_process),
                                      name=f'Comp_{str(next_to_process)}',
                                      proximal=proximalp,
                                      distal=distalp)

        nml_mor.segments.append(thisSeg)
        processed.append(next_to_process)

        available_points.remove(next_to_process)
        available_points += children[next_to_process]
        if not available_points:
            all_processed = True

    return nml_mor

In [27]:
def process_cables(segmentGroups, type_seg, nml_mor, nml_cell, errors):
    '''
    This function incorporates the segment groups into the morphology object and adds them to bigger segment groups.
    The morphology object is then added to the cell object.

    Input: - segmentGroups: list with lists of segmentgroups [[point], [point], ...]
           - type_seg: dict {point (int): type morph. part (e.g. soma) (str)}
           - nml_mor: neuroml morphology object
           - nml_cell: neuroml cell object
    
    Returns: nml_cell: neuroml cell object
    '''

    cablenumber = 1
    cables = {}
    type_cab = {}

    # Create main segment groups
    all_cables = neuroml.SegmentGroup(id='all')
    soma_group = neuroml.SegmentGroup(id='soma_group', neuro_lex_id='SAO:1044911821')
    axon_group = neuroml.SegmentGroup(id='axon_group', neuro_lex_id='SAO:1770195789')
    dendrite_group = neuroml.SegmentGroup(id='dendrite_group', neuro_lex_id='SAO:1211023249')
    basal_group = neuroml.SegmentGroup(id='basal_group', neuro_lex_id='SAO:1079900774')
    apical_group = neuroml.SegmentGroup(id='apical_group', neuro_lex_id='SAO:273773228')

    custom_groups = {}  # Dictionary to hold custom segment groups

    for segmentGroup in segmentGroups:
        type_cable = ''
        cable_id = f'{type_seg[segmentGroup[0]]}_{cablenumber}'
        this_cable = neuroml.SegmentGroup(id=cable_id, neuro_lex_id='SAO:864921383')

        for segment in reversed(segmentGroup):
            member = neuroml.Member(segments=segment)
            this_cable.members.append(member)
            type_this_seg = type_seg[segment]
            if type_cable and type_cable != type_this_seg:
                print(f"Error; cable {cablenumber} has multiple types!")
            else:
                type_cable = type_this_seg

        cables[cablenumber] = this_cable
        cable_include = neuroml.Include(segment_groups=cable_id)
        all_cables.includes.append(cable_include)

        if type_cable == 'soma':
            soma_group.includes.append(cable_include)
        elif type_cable == 'axon':
            axon_group.includes.append(cable_include)
        elif type_cable == 'bas_dend':
            basal_group.includes.append(cable_include)
            dendrite_group.includes.append(cable_include)
        elif type_cable == 'ap_dend':
            apical_group.includes.append(cable_include)
            dendrite_group.includes.append(cable_include)
        else:
            custom_group_id = f'{type_cable}_group'
            if custom_group_id not in custom_groups:
                custom_group = neuroml.SegmentGroup(id=custom_group_id)
                custom_groups[custom_group_id] = custom_group
            custom_groups[custom_group_id].includes.append(cable_include)

        type_cab[cablenumber] = type_cable
        cablenumber += 1

    # Check if cell contains soma segmentgroup
    if not soma_group.includes:
        log_error(errors, "No soma segments detected", fix="No fixes. SWC file is invalid.", stop=True)

    # Append all cables and segment groups to morphology
    for cable in cables.values():
        nml_mor.segment_groups.append(cable)

    for type in [all_cables, basal_group, apical_group, soma_group, axon_group, dendrite_group]:
        if type.includes:
            nml_mor.segment_groups.append(type)

    for custom_group in custom_groups.values():
        nml_mor.segment_groups.append(custom_group)

    nml_cell.morphology = nml_mor

    return nml_cell

In [28]:
def define_biophysical_properties(nml_cell, Cell_ID):
    '''
    This function defines some basic biophysical properties for the given cell.
    
    Input: - nml_cell: neuroml cell object
           - Cell_ID: unique ID of neuroml cell (str) 
    
    Returns: nml_cell: neuroml cell object
    '''

    # Create biophysical properties object
    all_props = neuroml.BiophysicalProperties(id=f'{Cell_ID}_properties')

    # Create and configure membrane properties
    membrane_props = neuroml.MembraneProperties()
    membrane_props.spike_threshes.append(neuroml.SpikeThresh(value='0.0 mV'))
    membrane_props.specific_capacitances.append(neuroml.SpecificCapacitance(value='1.0 uF_per_cm2'))
    membrane_props.init_memb_potentials.append(neuroml.InitMembPotential(value='-60.0 mV'))

    # Create and configure intracellular properties
    intra_props = neuroml.IntracellularProperties()
    intra_props.resistivities.append(neuroml.Resistivity(value='0.03 kohm_cm'))

    # Assign properties to the object
    all_props.membrane_properties = membrane_props
    all_props.intracellular_properties = intra_props

    # Assign object to cell
    nml_cell.biophysical_properties = all_props

    return nml_cell

In the cell below, you can specify the file to be converted, as well as the output directory in which the converted file will be stored.

In [29]:
path = 'swc_no_api\GGN_20170309_sc.swc'
output_dir = ''

swc_file = os.path.basename(path)
# try:
nml_file = construct_nml(path, output_dir='')
print(f'Converted {swc_file} to the following file: {nml_file}\n')


GGN_20170309_sc GGN_20170309_sc
{'Endpoint of zero radius detected': {'extra_info': ['Point 14249',
                                                     'Point 26425'],
                                      'fix': 'Changed radius to small number '
                                             '1e-06',
                                      'occurrences': 2},
 'Two segments detected with same radius and coordinates': {'extra_info': ['Segments '
                                                                           '910 '
                                                                           'and '
                                                                           '909',
                                                                           'Segments '
                                                                           '1436 '
                                                                           'and '
                                                        

**Neuromorpho API**

If you want to use the NeuroMorpho API to fetch SWC files from their website, you can use the code below.

*Warning: Currently, this code is causing issues with the NeuroMorpho.org website. The team is working on resolving this problem. Please avoid using this code for now.*

In [ ]:
# Necessary imports
from neuromorpho_api import requestor as requests

In [ ]:
def fetch_swc_file(neuron_id):
    '''
    This function fetches the information about the specified neuron id and fetches the corresponding SWC file using the generated url.

    Input: neuron_id: id of neuron on neuromorpho.org (int)

    Returns: SWC file contents
    '''

    endpoint = "https://neuromorpho.org/api/"
    response = requests.get(endpoint + f"neuron/id/{neuron_id}")

    if response.status_code != 200:
        raise("Failed to fetch SWC file:", response.text)

    data = response.json()

    # Construct and fetch the SWC URL
    swc_url = f"https://neuromorpho.org/dableFiles/{data['archive'].lower()}/CNG%20version/{data['neuron_name']}.CNG.swc"
    swc_response = requests.get(swc_url)
    
    if swc_response.status_code != 200:
        raise("Failed to fetch SWC file:", swc_response.text)

    return swc_response.content

In [ ]:
def create_swc_file(neuron_id, output_dir=''):
    '''
    This function writes the SWC contents to a new SWC file in an optionally specified output directory.

    Input: - neuron_id: id of neuron on neuromorpho.org (int)
           - output_dir (optional):  directory in which the SWC file will be saved (str)
    
    Returns: name of the newly created neuroml file (str)
    '''

    swc_content = fetch_swc_file(neuron_id)
    
    if swc_content:
        if output_dir:
            with open(f"{output_dir}/neuron_{neuron_id}.swc", "wb") as f:
                f.write(swc_content)

            return f"{output_dir}/neuron_{neuron_id}.swc"
        
        else:
            with open(f"neuron_{neuron_id}.swc", "wb") as f:
                f.write(swc_content)
                
            return f"neuron_{neuron_id}.swc"

In the cell below, you can specify what range of cells you want to fetch from the API, as well as the output directories in which the SWC and NML files will be stored.

In [ ]:
range_api = (1, 10)
output_dir_swc = ''
output_dir_nml = ''

for neuron_id in range(*range_api):
    swc_file = create_swc_file(neuron_id, output_dir=output_dir_swc)
    try:
        nml_file = convert_to_nml(swc_file, output_dir=output_dir_nml)
        print(f'Converted {swc_file.split('/')[-1]} to the following file: {nml_file}\n')
    except Exception as e:
        print(f'Error converting {swc_file}: {e}\n')